# Short-Circuit Calculation according to IEC 60909

pandapower supports short-circuit calculations with the equivalent source. The pandapower short-circuit calculation currently supports the following elements

- bus
- load (neglected)
- shunt (neglected)
- sgen
- gen
- ext_grid
- line
- trafo
- trafo3w
- switch

with the correction factors as defined in IEC 60909.

The following short-circuit currents can be calculated:
- ikss (Initial symmetrical short-circuit current)
- ip (short-circuit current peak)
- ith (equivalent thermal short-circuit current)

where ikss ip and ith are only implemented for networks without synchronous generators.

The results for all elements are tested against commercial software to ensure that correction factors are correctly applied. 

### Example Network

Here is a little example on how to use the short-circuit calculation. First, we create a simple line network with 4 buses, that are connected by one transformer and two lines. The network is fed by an external grid connection at bus 1:

In [59]:
import pandapower as pp

net = pp.create_empty_network()
b1 = pp.create_bus(net, vn_kv=110.)
b2 = pp.create_bus(net, vn_kv=20.)
b3 = pp.create_bus(net, vn_kv=20.)
b4 = pp.create_bus(net, vn_kv=20.)
pp.create_ext_grid(net, b1, s_sc_max_mva=100., s_sc_min_mva=40., rx_min=0.1, rx_max=0.1)
pp.create_transformer(net, b1, b2, std_type="63 MVA 110/20 kV")
pp.create_line(net, b2, b3, length_km=2., std_type="NA2XS2Y 1x95 RM/25 12/20 kV")
pp.create_line(net, b3, b4, length_km=2., std_type="NA2XS2Y 1x95 RM/25 12/20 kV")
net

This pandapower network includes the following parameter tables:
   - line (2 elements)
   - trafo (1 elements)
   - ext_grid (1 elements)
   - bus (4 elements)

### Maximum Initial Short-Circuit Current
Now, we import the shortcircuit submodule and calculate the maximum short-circuit currents with the runsc function:

In [60]:
import pandapower.shortcircuit as sc

sc.runsc(net, case="max")
net.res_bus_sc

,ikss_max_ka
0,0.524864
1,2.493060
2,2.331817
3,2.168514


### Other Short-Circuit Currents

Apart from the basic subtransient short-circuit current, we can also calculate thermal current ith and peak current ip:

In [61]:
sc.runsc(net, case="max", ith=True, ip=True)
net.res_bus_sc

,ikss_max_ka,ip_max_ka,ith_max_ka
0,0.524864,1.296004,0.533745
1,2.493060,6.232063,2.539753
2,2.331817,5.121186,2.351413
3,2.168514,4.331045,2.180716


### Minimum Short-Circuit Currents

Minimum short-circuits can be calculated in the same way. However, we need to specify the end temperature of the lines after a fault as per standard first:

In [62]:
net.line["endtemp_degree"] = 80
sc.runsc(net, case="min", ith=True, ip=True)
net.res_bus_sc

,ikss_min_ka,ip_min_ka,ith_min_ka
0,0.209946,0.518402,0.213498
1,1.079694,2.681600,1.098844
2,1.044364,2.390963,1.055189
3,1.007100,2.153310,1.014592


### Static Generators

Static generators are also considered according to the norm. We add one at bus b2:

In [63]:
pp.create_sgen(net, b2, p_kw=0, sn_kva=1e3)
net

This pandapower network includes the following parameter tables:
   - sgen (1 elements)
   - line (2 elements)
   - trafo (1 elements)
   - ext_grid (1 elements)
   - bus (4 elements)
 and the following results tables:
   - res_bus_sc (4 elements)

If we run the short-circuit calculation again, we can see that the currents increased due to the contribution of the inverteres to the short-circuit currents.

In [64]:
sc.runsc(net, case="max", ith=True, ip=True)
net.res_bus_sc

,ikss_max_ka,ip_max_ka,ith_max_ka
0,0.542095,1.338570,0.551268
1,2.588319,6.467238,2.636606
2,2.414313,5.278320,2.434175
3,2.238463,4.442232,2.250753


### Synchronous Generators

Synchronous generators can also be considered in the short-circuit calculation. We create one at bus 3:

In [65]:
pp.create_gen(net, b3, p_kw=0, vm_pu=1.0, cos_phi=0.8, vn_kv=22, sn_kva=5e3, xdss=0.2, rdss=0.005)
net

This pandapower network includes the following parameter tables:
   - sgen (1 elements)
   - line (2 elements)
   - trafo (1 elements)
   - ext_grid (1 elements)
   - bus (4 elements)
   - gen (1 elements)
 and the following results tables:
   - res_bus_sc (4 elements)

and run the short-circuit calculation again:

In [66]:
sc.runsc(net, case="max")
net.res_bus_sc

,ikss_max_ka
0,0.574612
1,2.770945
2,2.595212
3,2.395220


Once again, the short-circuit current increases due to the contribution of the generator (ip and ith are not yet supported for synchronous generators).